In [1]:
import cv2
import pandas as pd
import numpy as np
import time
from pyzbar.pyzbar import decode
from datetime import datetime
import math

def simulate_realtime_update(interval_sec=10, duration_sec=60):
    global df
    start_time = time.time()
    while (time.time() - start_time) < duration_sec:
        today = pd.Timestamp('2025-05-31')
        df = update_status(df, today)
        inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

        print(f"\n🕒 Updated @ {datetime.now().strftime('%H:%M:%S')} (System Date: {today.date()})")
        print(f"Fresh: {len(fresh_table)} | Discount: {len(discount_table)} | Donation: {len(donation_table)} | Expired: {len(expired_table)}")
        time.sleep(interval_sec)

# --- Load Initial Data ---
df = pd.read_csv("grocery_df.csv")

# Convert necessary date columns
for col in ['Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date']:
    df[col] = pd.to_datetime(df[col])

# --- Status Function ---
def get_status(row, today):
    if today < row['Discount_Date']:
        return 'Fresh'
    elif row['Discount_Date'] <= today < row['Donation_Date']:
        return 'Discount'
    elif row['Donation_Date'] <= today < row['Expiration_Date']:
        return 'Donation'
    else:
        return 'Expired'

def update_status(df, today):
    df['Status'] = df.apply(lambda row: get_status(row, today), axis=1)
    return df

# --- Table Generator ---
def generate_tables(df):
    inventory_table = df[['Product_ID', 'Product_Name', 'Catagory', 'Supplier_ID', 'Supplier_Name',
                          'Stock_Quantity', 'Reorder_Level', 'Reorder_Quantity', 'Unit_Price($)',
                          'Date_Received', 'Expiration_Date', 'Discount_Date', 'Donation_Date',
                          'Shelf_Life_(Days)', 'Shelf_Life_(Years)', 'Shelf_Life_Bin', 'Status']]

    fresh_table = inventory_table[df['Status'] == 'Fresh']
    discount_table = inventory_table[df['Status'] == 'Discount']
    donation_table = inventory_table[df['Status'] == 'Donation']
    expired_table = inventory_table[df['Status'] == 'Expired']

    return inventory_table, fresh_table, discount_table, donation_table, expired_table

def reduce_stock_by_qr(product_id):
    global df
    product_mask = df['Product_ID'] == product_id
    if not product_mask.any():
        print(f"❌ Product_ID {product_id} not found.")
        return
    df.loc[product_mask, 'Stock_Quantity'] -= 1
    if df.loc[product_mask, 'Stock_Quantity'].values[0] <= 0:
        df = df[~product_mask]
        print(f"🗑️ Product_ID {product_id} stock is 0 — removed from inventory.")
    else:
        print(f"🔻 Reduced stock for Product_ID {product_id}. Remaining: {df.loc[product_mask, 'Stock_Quantity'].values[0]}")

# --- Refresh all tables ---
def refresh_all_tables(current_date):
    global df, inventory_table, fresh_table, discount_table, donation_table, expired_table
    df = update_status(df, pd.Timestamp(current_date))
    inventory_table, fresh_table, discount_table, donation_table, expired_table = generate_tables(df)

# --- Add new product ---
def add_new_item_via_qr(data):
    global df
    # Convert dates
    data['Date_Received'] = pd.to_datetime(data['Date_Received'])
    data['Expiration_Date'] = pd.to_datetime(data['Expiration_Date'])
    data['Unit_Price($)'] = float(data['Unit_Price($)'])  # ensure float
    data['Discount_Price($)'] = round(data['Unit_Price($)'] * 0.7, 2)


    # Swap dates if needed
    if data['Expiration_Date'] < data['Date_Received']:
        data['Date_Received'], data['Expiration_Date'] = data['Expiration_Date'], data['Date_Received']

    # Shelf life
    shelf_life_days = (data['Expiration_Date'] - data['Date_Received']).days
    shelf_life_years = round(shelf_life_days / 365, 2)

    # Bin
    bin_label = '0-1 years' if shelf_life_years < 1 else f'{int(shelf_life_years)}-{int(shelf_life_years)+1} years'

    # Discount/Donation
    discount_fraction = 0.2 if bin_label == '0-1 years' else 0.1
    donation_fraction = 0.1 if bin_label == '0-1 years' else 0.05

    discount_date = data['Expiration_Date'] - pd.to_timedelta(shelf_life_days * discount_fraction, unit='D')
    donation_date = data['Expiration_Date'] - pd.to_timedelta(shelf_life_days * donation_fraction, unit='D')

    # Add columns
    data['Shelf_Life_(Days)'] = shelf_life_days
    data['Shelf_Life_(Years)'] = shelf_life_years
    data['Shelf_Life_Bin'] = bin_label
    data['Discount_Date'] = discount_date
    data['Donation_Date'] = donation_date

    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)
    print(f"✅ Added new item via QR: {data['Product_Name']}")

# --- QR Scanner ---
cap = cv2.VideoCapture(0)
print("Press 's' to scan QR code. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("QR Scanner", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        print("Exiting scanner...")
        break

    elif key == ord('s'):
        decoded = decode(frame)
        if not decoded:
            print("❌ No QR code found. Try again.")
            continue

        for barcode in decoded:
            qr_text = barcode.data.decode('utf-8')
            print("Scanned QR:", qr_text)

            try:
                # If only Product_ID is provided
                if qr_text.startswith("Product_ID:") and '|' not in qr_text:
                    product_id = qr_text.strip().split(':')[1]
                    reduce_stock_by_qr(product_id)
                    refresh_all_tables(current_date='2025-05-31')
                    continue


                # Else assume full product info
                scanned_info = dict(pair.split(':') for pair in qr_text.split('|'))
                scanned_info['Unit_Price'] = float(scanned_info.get('Unit_Price', '0').replace('$', '').strip())
                scanned_info['Stock_Quantity'] = int(scanned_info.get('Stock_Quantity', 0))
                scanned_info['Reorder_Level'] = int(scanned_info.get('Reorder_Level', 0))
                scanned_info['Reorder_Quantity'] = int(scanned_info.get('Reorder_Quantity', 0))
                scanned_info['Date_Received'] = pd.to_datetime(scanned_info['Date_Received'])
                scanned_info['Expiration_Date'] = pd.to_datetime(scanned_info['Expiration_Date'])

                if scanned_info not in df.to_dict('records'):
                    add_new_item_via_qr(scanned_info)
                    refresh_all_tables(current_date='2025-05-31')
                    print("✅ Product added and tables updated.")
                else:
                    print("⚠️ This QR code has already been added.")
            except Exception as e:
                print("❌ Error parsing QR:", e)


cap.release()
cv2.destroyAllWindows()

# Save after scanning
df.to_csv('inventory.csv', index=False)
print("✅ Table saved to inventory.csv")

# --- Start Real-Time Simulation ---
simulate_realtime_update(interval_sec=5, duration_sec=20)


Press 's' to scan QR code. Press 'q' to quit.
Scanned QR: Product_ID:28-044-4102
🔻 Reduced stock for Product_ID 28-044-4102. Remaining: 18
Scanned QR: Product_ID:40-690-9142|Product_Name:Pizza|Catagory:Frozen|Supplier_ID:69-933-2582|Supplier_Name:Realpoint|Stock_Quantity:1|Reorder_Level:90|Reorder_Quantity:99|Unit_Price($):0.80|Date_Received:2025-01-26|Expiration_Date:2025-06-05
✅ Added new item via QR: Pizza
✅ Product added and tables updated.
Exiting scanner...
✅ Table saved to inventory.csv

🕒 Updated @ 02:56:30 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 02:56:35 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 02:56:40 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565

🕒 Updated @ 02:56:45 (System Date: 2025-05-31)
Fresh: 323 | Discount: 56 | Donation: 46 | Expired: 565


In [2]:
df.tail()

,Product_ID,Product_Name,Catagory,Supplier_ID,Supplier_Name,Stock_Quantity,Reorder_Level,Reorder_Quantity,Unit_Price($),Date_Received,Expiration_Date,Shelf_Life_(Days),Shelf_Life_(Years),Shelf_Life_Bin,Discount_Date,Donation_Date,Discount_Price($),Status,Unit_Price
985,62-393-9939,Cheddar Cheese,Dairy,93-877-9384,Gabcube,60,9,89,9.0,2025-01-01,2025-05-05,126,0.35,0-1 years,2025-04-09 19:12:00,2025-04-22 09:36:00,6.30,Expired,NaN
986,31-745-6850,Cabbage,Fruits & Vegetables,96-215-2767,Lajo,94,90,12,0.9,2025-05-03,2025-06-01,29,0.08,0-1 years,2025-05-26 04:48:00,2025-05-29 02:24:00,0.63,Donation,NaN
987,86-692-2312,Avocado Oil,Oils & Fats,77-783-4107,Dazzlesphere,30,48,52,10.0,2024-11-30,2025-01-11,42,0.12,0-1 years,2025-01-02 14:24:00,2025-01-06 19:12:00,7.00,Expired,NaN
988,28-044-4102,Papaya,Fruits & Vegetables,93-358-1118,Yakijo,18,28,83,4.5,2025-03-27,2025-06-26,91,0.25,0-1 years,2025-06-07 19:12:00,2025-06-16 21:36:00,3.15,Fresh,NaN
989,40-690-9142,Pizza,Frozen,69-933-2582,Realpoint,1,90,99,0.8,2025-01-26,2025-06-05,130,0.36,0-1 years,2025-05-10 00:00:00,2025-05-23 00:00:00,0.56,Donation,0.0
